In [1]:
import sqlite3

In [2]:
! mkdir -p databases

In [3]:
DB_PATH = "databases/purchases.db"

In [4]:
connection = sqlite3.connect(DB_PATH)

## Insert

### Pure SQL Approach

In [5]:
cursor = connection.cursor()

# SQLite distinguishes INT PRIMARY KEY from INTEGER PRIMARY KEY...

cursor.execute('''
    CREATE TABLE Purchase(
        id INTEGER PRIMARY KEY,
        date TEXT,
        name TEXT,
        price REAL
    );
''')

cursor.execute('''
    INSERT INTO Purchase(date, name, price) VALUES
        ('2000-06-13', 'Harry Potter and the Philosopher''s Stone by J. K. Rowling', 200.00),
        ('2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.00),
        ('2008-06-12', 'Артур и минипуты, Люк Бессон', 150.00),
        ('2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.00),
        ('2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99);
''')

connection.commit()

### "Programmer's" Approach

In [6]:
cursor = connection.cursor()

purchases = [
    ('2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.00),
    ('2023-03-15', 'Swed House', 1500.00),
    ('2023-03-18', 'Вкусно – и точка', 500.00),
    ('2023-03-19', 'Funky Monkey Orange', 100.00),
    ('2023-03-19', 'Ластик с собачкой', 250.00),
]

cursor.executemany(
    """INSERT INTO Purchase(date, name, price)
       VALUES (?, ?, ?);""",
    purchases
)

connection.commit()

## Select

In [7]:
cursor = connection.cursor()

cursor.execute("SELECT * FROM Purchase WHERE name = 'Вкусно – и точка';")

print(cursor.fetchone())

(8, '2023-03-18', 'Вкусно – и точка', 500.0)


In [8]:
print(cursor.fetchone())

None


In [9]:
cursor.execute("SELECT * FROM Purchase WHERE name LIKE 'Harry Potter%';")

print(cursor.fetchall())

[(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0), (2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)]


In [10]:
for row in cursor.execute("SELECT * FROM Purchase ORDER BY price;"):
    print(row)

(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(10, '2023-03-19', 'Ластик с собачкой', 250.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(7, '2023-03-15', 'Swed House', 1500.0)


In [11]:
cursor.execute(
    "SELECT * FROM Purchase WHERE name = :product_name",
    {"product_name": 'Funky Monkey Orange'}
)

print(cursor.fetchone())

(9, '2023-03-19', 'Funky Monkey Orange', 100.0)


## Update, Delete

In [12]:
cursor = connection.cursor()

for row in cursor.execute("SELECT * FROM Purchase;"):
    print(row)

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 250.0)


In [13]:
def select_all(connection, table_name: str) -> None:
    cursor = connection.cursor()

    for row in cursor.execute(f"SELECT * FROM {table_name};"):
        print(row)

In [14]:
column_names = [
    description[0] for description in cursor.description
]

In [15]:
print(column_names)

['id', 'date', 'name', 'price']


### Update and Rollback

In [16]:
cursor.execute(
    "UPDATE Purchase SET price = price * 1.5 WHERE name = 'Ластик с собачкой';"
)

In [17]:
select_all(connection, 'Purchase')

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 375.0)


Another connection does not see changes if they are not committed.

In [18]:
connection2 = sqlite3.connect(DB_PATH)
cursor2 = connection2.cursor()

for row in cursor2.execute("SELECT * FROM Purchase;"):
    print(row)

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 250.0)


In [19]:
for row in cursor2.execute("SELECT * FROM Purchase;"):
    print(row)

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 250.0)


Undoing changes:

In [20]:
connection.rollback()

In [21]:
select_all(connection, 'Purchase')

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 250.0)


In [29]:
connection2.close()

### Update, Delete, and Rollback

In [22]:
cursor.execute(
    "UPDATE Purchase SET price = price * 2 WHERE name = 'Ластик с собачкой';"
)

In [23]:
select_all(connection, 'Purchase')

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 500.0)


In [24]:
cursor.execute("DELETE FROM Purchase WHERE name = 'Артур и минипуты, Люк Бессон';")

In [25]:
select_all(connection, 'Purchase')

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 500.0)


In [26]:
connection.rollback()

In [27]:
for row in cursor.execute("SELECT * FROM Purchase;"):
    print(row)

(1, '2000-06-13', "Harry Potter and the Philosopher's Stone by J. K. Rowling", 200.0)
(2, '2001-06-20', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 200.0)
(3, '2008-06-12', 'Артур и минипуты, Люк Бессон', 150.0)
(4, '2016-09-15', 'Методы решения задач в общем курсе физики (Механика), Корявов В.П.', 250.0)
(5, '2023-02-09', 'Love Will Tear Us Apart by C.K. McDonnell', 14.99)
(6, '2023-03-01', 'Электричка (Тимирязевская <-> Новодачная)', 64.0)
(7, '2023-03-15', 'Swed House', 1500.0)
(8, '2023-03-18', 'Вкусно – и точка', 500.0)
(9, '2023-03-19', 'Funky Monkey Orange', 100.0)
(10, '2023-03-19', 'Ластик с собачкой', 250.0)


In [28]:
connection.close()

## Context Manager To Do Things Without Commit/Rollback

In [33]:
connection = sqlite3.connect(":memory:")


connection.execute("""
    CREATE TABLE Person(
        id INTEGER PRIMARY KEY,
        firstname VARCHAR UNIQUE
    );
""")


with connection:
    connection.execute(
        "INSERT INTO Person(firstname) VALUES (?);", ("Joe",)
    )

with connection:
    cursor = connection.execute(
        "SELECT * FROM Person;"
    )
    
    print(cursor.fetchall())


try:
    with connection:
        connection.execute(
            "INSERT INTO Person(firstname) VALUES (?);", ("Joe",)
        )
except sqlite3.IntegrityError:
    print("Can't add Joe twice!")


connection.close()

[(1, 'Joe')]
Can't add Joe twice!


## "Programmer's" SELECT

In [34]:
connection = sqlite3.connect(DB_PATH)
connection.row_factory = sqlite3.Row

cursor = connection.cursor()

cursor.execute("SELECT * FROM Purchase WHERE name LIKE 'Harry%';")
row = cursor.fetchone()

In [35]:
row

In [36]:
print(row.keys())

['id', 'date', 'name', 'price']


In [37]:
for key in row.keys():
    print(row[key])

1
2000-06-13
Harry Potter and the Philosopher's Stone by J. K. Rowling
200.0


In [38]:
dict(row)

{'id': 1,
 'date': '2000-06-13',
 'name': "Harry Potter and the Philosopher's Stone by J. K. Rowling",
 'price': 200.0}

In [39]:
# https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.row_factory

class DictFactory:
    def __call__(self, cursor, row):
        result = dict()
        
        for index, column_data in enumerate(cursor.description):
            column_name = column_data[0]
            result[column_name] = row[index]
        
        return result

In [40]:
connection.row_factory = DictFactory()

cursor = connection.cursor()

cursor.execute("SELECT * FROM Purchase WHERE name LIKE 'Harry%';")
row = cursor.fetchone()

In [41]:
row

{'id': 1,
 'date': '2000-06-13',
 'name': "Harry Potter and the Philosopher's Stone by J. K. Rowling",
 'price': 200.0}

In [42]:
connection.close()

# PyMongo

https://pymongo.readthedocs.io/en/stable/tutorial.html

In [43]:
from pymongo import MongoClient

```bash
# Install MongoDB ...

sudo mongod
```

![](./images/MongoRunning0.jpg)
![](./images/MongoRunning.jpg)

In [44]:
client = MongoClient('localhost', 27017)

In [45]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [66]:
db = client.the_lord_of_the_rings

In [67]:
collection = db.characters

In [68]:
collection.insert_one({
    'name': 'Dopey',
    'race': 'dwarf'
})

In [69]:
db.list_collection_names()

['characters']

In [70]:
collection.insert_one({
    'name': 'Legolas',
    'race': 'elf'
})

In [71]:
collection.find_one()

{'_id': ObjectId('64176852ec5cc6de05f96033'), 'name': 'Dopey', 'race': 'dwarf'}

In [72]:
collection.find_one({'race': 'elf'})

{'_id': ObjectId('64176852ec5cc6de05f96034'), 'name': 'Legolas', 'race': 'elf'}

In [73]:
collection.count_documents({})

2

In [74]:
collection.update_one(
    {'name': 'Legolas'},
    {'$set': {'age': 3000}}
)

In [75]:
collection.find_one({'race': 'elf'})

{'_id': ObjectId('64176852ec5cc6de05f96034'),
 'name': 'Legolas',
 'race': 'elf',
 'age': 3000}